# ETL pipeline for the Tableau REST API

In [1]:
"""
This Jupyter notebook will be the code that is expected to run the ETL data pipeline
for building the database for the Tableau REST API.

Dependencies: pymysql, requests, dotenv

How to install dependencies: 
run 'pip3 install pymysql requests python-dotenv' or 'pip install pymysql requests python-dotenv'
"""
__authors__ = ["Mustafa Aljumayli"]
%run env.py
import pymysql
import requests
import time
import xml.etree.ElementTree as ET
from datetime import datetime
from env import getenv

start_time = time.time()



### Establishing the connection to the database

In [2]:
"""
To access our database we must first connect to it. The pymysql library allows us
to define this connection so that we may later begin to write queries on our database.
"""

sqlconnection = pymysql.connect(
    host=getenv('MYSQL_HOST'),
    user=getenv('MYSQL_USER'),
    password=getenv('MYSQL_PASSWORD'),
    port=int(getenv('MYSQL_PORT')),
    database=getenv('MYSQL_DATABASE')
)


### Obtain Tableau API Token

In [3]:
"""
To run the Tableau REST API, you must authenticate yourself to Tableau Server
so you can grab the items you need. So we send a post request, parse the response,
assign into a variable that we can inject elsewhere in our program.
"""
# Obtain API Token
def get_tableau_token(username, password, content_url):
    api_url = getenv('TABLEAU_API_URL')
    url = f"{api_url}/auth/signin"
    payload = f"""
    <tsRequest>
        <credentials name="{username}" password="{password}">
            <site contentUrl="{content_url}" />
        </credentials>
    </tsRequest>
    """
    headers = {
        "Content-Type": "application/xml"
    }
    response = requests.post(url, headers=headers, data=payload)
    
    # Parsing the response
    if response.status_code != 200:
        raise Exception(f"Failed to log in, status code: {response.status_code}, response: {response.text}")
    root = ET.fromstring(response.content)
    
    site_id = root.find('.//t:site', namespaces={'t': 'http://tableau.com/api'}).get('id')
    token = root.find('.//t:credentials', namespaces={'t': 'http://tableau.com/api'}).attrib['token']
    return token, site_id

# Get the token
username, password, content_url = getenv('TABLEAU_USERNAME'), getenv('TABLEAU_PASSWORD'), getenv('TABLEAU_CONTENT_URL')
token, site_id = get_tableau_token(username, password, content_url)

print("Authentication successful. Token retrieved.")


Authentication successful. Token retrieved.


### Test Cells

In [4]:
"""
This cell is meant to help test the connection and show you the tables that are
in the database.
"""
db = getenv('MYSQL_DATABASE')
with sqlconnection.cursor() as cursor:
    cursor.execute(f"SHOW TABLES IN {db}")
    tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])


Tables in the database:
connections_list
datasource_list
workbook_list


In [5]:
"""
This cell is meant to help test the connection and show you the contents of each
table in the database.
"""

with sqlconnection.cursor() as cursor:
    cursor.execute(f"SHOW TABLES IN {db}")
    tables = cursor.fetchall()

    print("Tables in the database:")
    for table in tables:
        table_name = table[0]
        print(f"\nContents of table {table_name}:")
        cursor.execute(f"SELECT * FROM {db}.{table_name}")
        rows = cursor.fetchall()
        for row in rows:
            print(row)


Tables in the database:

Contents of table connections_list:
('066ace17-6d63-4986-8175-d2f9020554b1_aea842c8-b8a4-46a8-9dba-a2cacf1f6cca', 0, None, None, None, 'e772b085-d3cb-4c16-90a0-de93de740fcc', 'Master_Course_Document.xlsx', None, '066ace17-6d63-4986-8175-d2f9020554b1', 'onedrive_master_program_list_new_updated_connection_live', 'aea842c8-b8a4-46a8-9dba-a2cacf1f6cca', 'Program Manager Event Dashboard (for PM use)', datetime.datetime(2024, 7, 23, 10, 35, 54), datetime.datetime(2024, 7, 25, 6, 21, 38))
('066ace17-6d63-4986-8175-d2f9020554b1_bb95319b-423e-469b-8271-3a752a188010', 0, None, None, None, 'e772b085-d3cb-4c16-90a0-de93de740fcc', 'Master_Course_Document.xlsx', None, '066ace17-6d63-4986-8175-d2f9020554b1', 'onedrive_master_program_list_new_updated_connection_live', 'bb95319b-423e-469b-8271-3a752a188010', 'test', datetime.datetime(2024, 7, 23, 10, 35, 54), datetime.datetime(2024, 7, 25, 6, 21, 38))
('0705f4a6-1a52-4539-8212-1373ce624553_01f3834c-8e83-4013-ba9d-aeed71bfccbc',

### Create Tables

In [6]:
"""
This method defines the tables we need in our database if they haven't
been defined already.
"""
def create_tables():
    with sqlconnection.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS workbook_list (
                id VARCHAR(255) PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                description TEXT,
                content_url VARCHAR(255) NOT NULL,
                webpage_url VARCHAR(255),
                show_tabs BOOLEAN,
                size INT,
                created_at DATETIME NOT NULL,
                updated_at DATETIME NOT NULL,
                encrypt_extracts BOOLEAN,
                default_view_id VARCHAR(255),
                project_id VARCHAR(255) NOT NULL,
                project_name VARCHAR(255),
                owner_id VARCHAR(255) NOT NULL,
                owner_name VARCHAR(255),
                hits_total INT,
                hits_last_two_weeks_total INT,
                hits_last_one_month_total INT,
                hits_last_three_months_total INT,
                hits_last_twelve_months_total INT,
                date_extracted DATETIME NOT NULL,
                last_updated DATETIME NOT NULL
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS datasource_list (
                id VARCHAR(255) PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                description TEXT,
                content_url VARCHAR(255) NOT NULL,
                webpage_url VARCHAR(255),
                size INT,
                created_at DATETIME NOT NULL,
                updated_at DATETIME NOT NULL,
                encrypt_extracts BOOLEAN,
                project_id VARCHAR(255) NOT NULL,
                project_name VARCHAR(255),
                owner_id VARCHAR(255) NOT NULL,
                owner_name VARCHAR(255),
                has_extracts BOOLEAN,
                is_certified BOOLEAN,
                use_remote_query_agent BOOLEAN,
                hits_total INT,
                hits_last_two_weeks_total INT,
                hits_last_one_month_total INT,
                hits_last_three_months_total INT,
                hits_last_twelve_months_total INT,
                connected_workbooks INT NOT NULL,
                date_extracted DATETIME NOT NULL,
                last_updated DATETIME NOT NULL
            )
        """)
        """ 
        connection_id is a made up primary key because we chose to not use the REST API for this. 
        datasource_ids with the REST API for workbook connections differ than the ids in datasource_list.
        The metadata API provides luids which are synonymous with the ids in the datasource_list table.
        """
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS connections_list (
                connection_id VARCHAR(255) PRIMARY KEY,
                is_certified BOOLEAN,
                certifier_id VARCHAR(255),
                certifier_name VARCHAR(255),
                certified_display_name VARCHAR(255),
                database_id VARCHAR(255) NOT NULL,
                database_name VARCHAR(255) NOT NULL,
                extract_last_refresh_time DATETIME,
                datasource_id VARCHAR(255) NOT NULL,
                datasource_name VARCHAR(255) NOT NULL,
                workbook_id VARCHAR(255) NOT NULL,
                workbook_name VARCHAR(255) NOT NULL,
                FOREIGN KEY (datasource_id) REFERENCES datasource_list(id),
                FOREIGN KEY (workbook_id) REFERENCES workbook_list(id),
                date_extracted DATETIME NOT NULL,
                last_updated DATETIME NOT NULL
            )
        """)
        sqlconnection.commit()

# Create tables
create_tables()


## Extraction & Transoformation

#### Extraction of data for the workbook_list table

In [8]:
"""
This cell is meant to fetch all workbooks and then parse the
response such that we can create a list of workbooks where 
each workbook gets recorded. The extraction happens from the API response, 
and the transformation comes from turning parsed XML to a hashmap of data.
"""

def parse_datetime(dt_str):
    return datetime.strptime(dt_str, "%Y-%m-%dT%H:%M:%SZ")

def parse_bool(s):
    return s.lower() == 'true'

# Retrieves workbook data from API
def get_workbooks(token, site_id):
    api_url = getenv('TABLEAU_API_URL')
    workbooks_url = f"{api_url}/sites/{site_id}/workbooks?pageSize=1000&pageNumber=1"
    headers = {"X-Tableau-Auth": token}
    response = requests.get(workbooks_url, headers=headers)
    
    # Parses workbook data
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        workbooks = []
        for workbook in root.findall('.//t:workbook', namespaces={'t': 'http://tableau.com/api'}):
            project = workbook.find('.//t:project', namespaces={'t': 'http://tableau.com/api'})
            owner = workbook.find('.//t:owner', namespaces={'t': 'http://tableau.com/api'})
            
            created_at = parse_datetime(workbook.get('createdAt')) if workbook.get('createdAt') else None
            updated_at = parse_datetime(workbook.get('updatedAt')) if workbook.get('updatedAt') else None
            
            workbook_data = {
                'id': workbook.get('id'),
                'name': workbook.get('name'),
                'description': workbook.get('description', ''),
                'content_url': workbook.get('contentUrl', ''),
                'webpage_url': workbook.get('webpageUrl', ''),
                'show_tabs': parse_bool(workbook.get('showTabs', 'false')),
                'size': int(workbook.get('size', 0)),
                'created_at': created_at.isoformat() if created_at else None,
                'updated_at': updated_at.isoformat() if updated_at else None,
                'encrypt_extracts': parse_bool(workbook.get('encryptExtracts', 'false')),
                'default_view_id': workbook.get('defaultViewId', ''),
                'project_id': project.get('id') if project is not None else '',
                'project_name': project.get('name') if project is not None else '',
                'owner_id': owner.get('id') if owner is not None else '',
                'owner_name': owner.get('name') if owner is not None else ''
            }
            workbooks.append(workbook_data)
        return workbooks
    else:
        raise Exception(f"Failed to fetch workbooks: {response.status_code} {response.text}")

# Fetch all workbooks
workbooks = get_workbooks(token, site_id)
print(workbooks)
print(f"Fetched {len(workbooks)} workbooks.")


[{'id': 'e1971af7-e976-4950-a99b-60a6a0121d68', 'name': 'Event Dashboard Live v3', 'description': '', 'content_url': 'EventDashboardLivev3', 'webpage_url': 'http://tableau.unc.edu/#/site/sog/workbooks/1051', 'show_tabs': False, 'size': 1, 'created_at': '2020-04-10T00:55:21', 'updated_at': '2021-08-20T14:27:11', 'encrypt_extracts': False, 'default_view_id': '1a658248-5e5c-415d-9670-7aaef2c9e26c', 'project_id': '2c2547d6-52c4-4521-974d-9a2ea652c0a5', 'project_name': 'Broken Dashboards', 'owner_id': 'c042d4fe-baeb-4612-b7e4-416ccc56a78e', 'owner_name': 'laurengp'}, {'id': '4c6ba604-39c6-4cc9-bfc8-76636fadf1c5', 'name': 'Test', 'description': 'Test2', 'content_url': 'Test', 'webpage_url': 'http://tableau.unc.edu/#/site/sog/workbooks/1206', 'show_tabs': False, 'size': 1, 'created_at': '2020-05-15T18:06:34', 'updated_at': '2021-08-03T21:06:26', 'encrypt_extracts': False, 'default_view_id': '5686bef0-a59e-4b54-88cd-3697d1987574', 'project_id': 'bd89433e-46d1-403f-8001-403e1be300f9', 'project_

In [9]:
"""
This cell is meant to get usage statistics for each workbook and then parse the
response such that we can add the usage statistics to their respective entry.
The extraction happens from the API response, and the transformation comes from 
turning parsed XML to a hashmap of data.
"""
# Retrieves workbook hits data from API
def get_wb_usage_statistics(token, workbook_id):
    server_url = getenv('TABLEAU_SERVER_URL')
    usage_url = f"{server_url}/api/-/content/usage-stats/workbooks/{workbook_id}"
    headers = {"X-Tableau-Auth": token}
    response = requests.get(usage_url, headers=headers)
    
    # Parses workbook hits
    if response.status_code == 200:
        usage_stats = response.json()
        return {
            "hits_total": int(usage_stats.get('hitsTotal', 0)),
            "hits_last_two_weeks_total": int(usage_stats.get('hitsLastTwoWeeksTotal', 0)),
            "hits_last_one_month_total": int(usage_stats.get('hitsLastOneMonthTotal', 0)),
            "hits_last_three_months_total": int(usage_stats.get('hitsLastThreeMonthsTotal', 0)),
            "hits_last_twelve_months_total": int(usage_stats.get('hitsLastTwelveMonthsTotal', 0))
        }
    else:
        raise Exception(f"Failed to fetch usage statistics for workbook {workbook_id}: {response.status_code} {response.text}")

# Fetch usage statistics for all workbooks
workbooks_with_usage = []
for workbook in workbooks:
    workbook_id = workbook['id']
    usage_stats = get_wb_usage_statistics(token, workbook_id)
    workbook['usage_stats'] = usage_stats
    workbook['last_updated'] = datetime.now().isoformat()
    if 'date_extracted' not in workbook:
        workbook['date_extracted'] = datetime.now().isoformat()  # This should only be set once when the workbook is first created
    workbooks_with_usage.append(workbook)

print(workbooks_with_usage)
print('Operation Successful')

def get_workbook_data():
    return workbooks_with_usage

workbook_data = get_workbook_data()
print("Workbook data prepared for database insertion.")


[{'id': 'e1971af7-e976-4950-a99b-60a6a0121d68', 'name': 'Event Dashboard Live v3', 'description': '', 'content_url': 'EventDashboardLivev3', 'webpage_url': 'http://tableau.unc.edu/#/site/sog/workbooks/1051', 'show_tabs': False, 'size': 1, 'created_at': '2020-04-10T00:55:21', 'updated_at': '2021-08-20T14:27:11', 'encrypt_extracts': False, 'default_view_id': '1a658248-5e5c-415d-9670-7aaef2c9e26c', 'project_id': '2c2547d6-52c4-4521-974d-9a2ea652c0a5', 'project_name': 'Broken Dashboards', 'owner_id': 'c042d4fe-baeb-4612-b7e4-416ccc56a78e', 'owner_name': 'laurengp', 'usage_stats': {'hits_total': 76, 'hits_last_two_weeks_total': 0, 'hits_last_one_month_total': 0, 'hits_last_three_months_total': 0, 'hits_last_twelve_months_total': 3}, 'last_updated': '2024-07-25T14:42:58.571445', 'date_extracted': '2024-07-25T14:42:58.571445'}, {'id': '4c6ba604-39c6-4cc9-bfc8-76636fadf1c5', 'name': 'Test', 'description': 'Test2', 'content_url': 'Test', 'webpage_url': 'http://tableau.unc.edu/#/site/sog/workboo

#### Extraction of data for the data_source_list table

In [10]:
"""
This cell is meant to fetch all data sources and then parse the
response such that we can create a list of data sources where 
each data source gets recorded. The extraction happens from the API call, 
and the transformation comes from turning parsed XML to a hashmap of data.
"""

# Retrieves datasources data from API
def get_datasources(token, site_id):
    api_url = getenv('TABLEAU_API_URL')
    endpoint = f"{api_url}/sites/{site_id}/datasources?pageSize=1000&pageNumber=1"
    headers = {"X-Tableau-Auth": token}
    response = requests.get(endpoint, headers=headers)
    
    # Parses datasource data from API
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        datasources = []
        # Parses the XML output
        for datasource in root.findall('.//t:datasource', namespaces={'t': 'http://tableau.com/api'}):
            created_at = parse_datetime(datasource.get('createdAt')) if datasource.get('createdAt') else None
            updated_at = parse_datetime(datasource.get('updatedAt')) if datasource.get('updatedAt') else None
            
            project = datasource.find('.//t:project', namespaces={'t': 'http://tableau.com/api'})
            owner = datasource.find('.//t:owner', namespaces={'t': 'http://tableau.com/api'})
            
            datasource_data = {
                'id': datasource.get('id'),
                'name': datasource.get('name'),
                'content_url': datasource.get('contentUrl', ''),
                'description': datasource.get('description', ''),
                'encrypt_extracts': datasource.get('encryptExtracts', ''),
                'has_extracts': parse_bool(datasource.get('encryptExtracts', 'false')),
                'is_certified': parse_bool(datasource.get('isCertified', 'false')),
                'size': int(datasource.get('size', 0)),
                'created_at': created_at.isoformat() if created_at else None,
                'updated_at': updated_at.isoformat() if updated_at else None,
                'use_remote_query_agent': parse_bool(datasource.get('useRemoteQueryAgent', 'false')),
                'webpage_url': datasource.get('webpageUrl', ''),
                'project_id': project.get('id') if project is not None else '',
                'project_name': project.get('name') if project is not None else '',
                'owner_id': owner.get('id') if owner is not None else '',
                'owner_name': owner.get('name') if owner is not None else '',
                'last_updated': datetime.now().isoformat(),
                'date_extracted': datetime.now().isoformat()
            }
            datasources.append(datasource_data)
        return datasources
    else:
        raise Exception(f"Failed to fetch datasources: {response.status_code} {response.text}")

datasources = get_datasources(token, site_id)
print(datasources)
print(f"Fetched {len(datasources)} datasources.")


[{'id': '165a74eb-46b6-40dd-87d7-29e348616d3e', 'name': '(Alumni Map)', 'content_url': 'AlumniMap', 'description': 'MPA Alumni', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_certified': False, 'size': 1, 'created_at': '2020-05-19T18:28:05', 'updated_at': '2022-04-29T19:03:44', 'use_remote_query_agent': False, 'webpage_url': '', 'project_id': 'dc16c7d5-fb0b-4a31-921a-32f74cbad584', 'project_name': 'Data Sources', 'owner_id': 'b359c294-54e8-44e8-a2a9-69dcb53f5448', 'owner_name': 'cararob', 'last_updated': '2024-07-25T14:43:43.301876', 'date_extracted': '2024-07-25T14:43:43.301876'}, {'id': 'be1bd066-a893-4d7f-a4ac-d56db09d04b4', 'name': 'unc_efc', 'content_url': 'unc_efc', 'description': 'EFC Dashboards - Staging Environment', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_certified': False, 'size': 1, 'created_at': '2020-09-23T03:22:42', 'updated_at': '2021-05-18T23:23:39', 'use_remote_query_agent': False, 'webpage_url': '', 'project_id': '77ed9f59-f68e-4bb0-9223-656

#### Get datasource hits and append it the usage stats to each entry

In [11]:
"""
This cell is meant to get usage statistics for each datasource and then parse the
response such that we can add the usage statistics to their respective entry.
The extraction happens from the API response, and the transformation comes from 
turning parsed XML to a hashmap of data.
"""

# Retrieves datasource hits data from API
def get_ds_usage_statistics(token, datasource_id):
    server_url = getenv('TABLEAU_SERVER_URL')
    usage_url = f"{server_url}/api/-/content/usage-stats/datasources/{datasource_id}"
    headers = {"X-Tableau-Auth": token}
    response = requests.get(usage_url, headers=headers)
    # Parses datasources hits data from API
    if response.status_code == 200:
        usage_stats = response.json()
        return {
            "hits_total": int(usage_stats.get('hitsTotal', 0)),
            "hits_last_two_weeks_total": int(usage_stats.get('hitsLastTwoWeeksTotal', 0)),
            "hits_last_one_month_total": int(usage_stats.get('hitsLastOneMonthTotal', 0)),
            "hits_last_three_months_total": int(usage_stats.get('hitsLastThreeMonthsTotal', 0)),
            "hits_last_twelve_months_total": int(usage_stats.get('hitsLastTwelveMonthsTotal', 0))
        }
    else:
        raise Exception(f"Failed to fetch usage statistics for datasource {datasource_id}: {response.status_code} {response.text}")
    
# Appends the hits fields for each datasource.
datasources_with_usage = []
for datasource in datasources:
    datasource_id = datasource['id']
    usage_stats = get_ds_usage_statistics(token, datasource_id)
    datasource['usage_stats'] = usage_stats
    datasources_with_usage.append(datasource)

print(datasources_with_usage)
print('Operation Successful')

def get_datasource_data():
    return datasources_with_usage

datasource_data = get_datasource_data()
print("Datasource data prepared for database insertion.")




[{'id': '165a74eb-46b6-40dd-87d7-29e348616d3e', 'name': '(Alumni Map)', 'content_url': 'AlumniMap', 'description': 'MPA Alumni', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_certified': False, 'size': 1, 'created_at': '2020-05-19T18:28:05', 'updated_at': '2022-04-29T19:03:44', 'use_remote_query_agent': False, 'webpage_url': '', 'project_id': 'dc16c7d5-fb0b-4a31-921a-32f74cbad584', 'project_name': 'Data Sources', 'owner_id': 'b359c294-54e8-44e8-a2a9-69dcb53f5448', 'owner_name': 'cararob', 'last_updated': '2024-07-25T14:43:43.301876', 'date_extracted': '2024-07-25T14:43:43.301876', 'usage_stats': {'hits_total': 4, 'hits_last_two_weeks_total': 0, 'hits_last_one_month_total': 0, 'hits_last_three_months_total': 0, 'hits_last_twelve_months_total': 0}}, {'id': 'be1bd066-a893-4d7f-a4ac-d56db09d04b4', 'name': 'unc_efc', 'content_url': 'unc_efc', 'description': 'EFC Dashboards - Staging Environment', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_certified': False, 'size': 1,

#### Gets the connections for each datasource and populates connections_list

In [12]:
# Retrieves connections data from Tableau Metadata API
def get_connections(token):
    server_url = getenv('TABLEAU_SERVER_URL')
    endpoint = f"{server_url}/api/metadata/graphql"
    headers = {'X-Tableau-Auth': token}
    query = {
        "query": """
        query published_datasources {
            publishedDatasources {
                luid
                name
                extractLastRefreshTime
                downstreamWorkbooks {
                    luid
                    name 
                }
                upstreamDatabases {
                    luid
                    name
                }
                isCertified
                certifier {
                    luid
                    name
                }
                certifierDisplayName
            }
        }
        """
    }
    
    response = requests.post(endpoint, headers=headers, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch connections: {response.status_code} {response.text}")

# Fetch connections using the provided query
connections_response = get_connections(token)

# Parses connections data from API and updates datasource_list table.
def parse_connections_data(connections_response):
    connections_list = []
    datasource_connections_count = {}
    
    for datasource in connections_response['data']['publishedDatasources']:
        datasource_id = datasource['luid']
        downstream_workbooks = datasource['downstreamWorkbooks']
        
        # Update datasource connections count
        datasource_connections_count[datasource_id] = len(downstream_workbooks)
        
        for workbook in downstream_workbooks:
            database_id = datasource['upstreamDatabases'][0]['luid'] if datasource['upstreamDatabases'] else None
            database_name = datasource['upstreamDatabases'][0]['name'] if datasource['upstreamDatabases'] else None
            
            # Only append if database_id is not null
            if database_id is not None:
                connection_data = {
                    'connection_id': f"{datasource_id}_{workbook['luid']}",
                    'is_certified': datasource['isCertified'],
                    'certifier_id': datasource['certifier']['luid'] if datasource.get('certifier') else None,
                    'certifier_name': datasource['certifier']['name'] if datasource.get('certifier') else None,
                    'certified_display_name': datasource.get('certifierDisplayName', None),
                    'database_id': database_id,
                    'database_name': database_name,
                    'extract_last_refresh_time': datasource['extractLastRefreshTime'],
                    'datasource_id': datasource_id,
                    'datasource_name': datasource['name'],
                    'workbook_id': workbook['luid'],
                    'workbook_name': workbook['name'],
                    'last_updated': datetime.now().isoformat(),
                    'date_extracted': datetime.now().isoformat()
                }
                connections_list.append(connection_data)
    
    return connections_list, datasource_connections_count

# Function to update datasource list with connections count
def update_datasource_list(datasources, datasource_connections_count):
    updated_datasources = []
    for datasource in datasources:
        datasource_id = datasource['id']
        datasource['connected_workbooks'] = datasource_connections_count.get(datasource_id, 0)
        updated_datasources.append(datasource)
    return updated_datasources

# Parse the connections data
connections_data, datasource_connections_count = parse_connections_data(connections_response)

# Update the datasource list with connections count
datasources_with_usage = update_datasource_list(datasources_with_usage, datasource_connections_count)
# Insert data into the connections_list table
print(datasources_with_usage)
connects = connections_data 
print(connects)
print(f"Updated datasource list with connected workbooks count and parsed {len(connects)} connection entries.")

[{'id': '165a74eb-46b6-40dd-87d7-29e348616d3e', 'name': '(Alumni Map)', 'content_url': 'AlumniMap', 'description': 'MPA Alumni', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_certified': False, 'size': 1, 'created_at': '2020-05-19T18:28:05', 'updated_at': '2022-04-29T19:03:44', 'use_remote_query_agent': False, 'webpage_url': '', 'project_id': 'dc16c7d5-fb0b-4a31-921a-32f74cbad584', 'project_name': 'Data Sources', 'owner_id': 'b359c294-54e8-44e8-a2a9-69dcb53f5448', 'owner_name': 'cararob', 'last_updated': '2024-07-25T14:43:43.301876', 'date_extracted': '2024-07-25T14:43:43.301876', 'usage_stats': {'hits_total': 4, 'hits_last_two_weeks_total': 0, 'hits_last_one_month_total': 0, 'hits_last_three_months_total': 0, 'hits_last_twelve_months_total': 0}, 'connected_workbooks': 0}, {'id': 'be1bd066-a893-4d7f-a4ac-d56db09d04b4', 'name': 'unc_efc', 'content_url': 'unc_efc', 'description': 'EFC Dashboards - Staging Environment', 'encrypt_extracts': 'false', 'has_extracts': False, 'is_cer

## Loading methods to populate tables

### Inserting into workbook_list

In [13]:
"""
Insert the collected data into the SQL tables.
"""
# Method to insert data into the workbook_list table
def insert_workbooks(sqlconnection, workbooks):
    with sqlconnection.cursor() as cursor:
        for workbook in workbooks:
            cursor.execute("""
            INSERT INTO workbook_list (
                id, 
                name, 
                description, 
                content_url, 
                webpage_url, 
                show_tabs, 
                size, 
                created_at, 
                updated_at,
                encrypt_extracts, 
                default_view_id, 
                project_id, 
                project_name, 
                owner_id, 
                owner_name, 
                hits_total,
                hits_last_two_weeks_total, 
                hits_last_one_month_total, 
                hits_last_three_months_total,
                hits_last_twelve_months_total, 
                date_extracted, 
                last_updated
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                name = VALUES(name),
                description = VALUES(description),
                content_url = VALUES(content_url),
                webpage_url = VALUES(webpage_url),
                show_tabs = VALUES(show_tabs),
                size = VALUES(size),
                created_at = VALUES(created_at),
                updated_at = VALUES(updated_at),
                encrypt_extracts = VALUES(encrypt_extracts),
                default_view_id = VALUES(default_view_id),
                project_id = VALUES(project_id),
                project_name = VALUES(project_name),
                owner_id = VALUES(owner_id),
                owner_name = VALUES(owner_name),
                hits_total = VALUES(hits_total),
                hits_last_two_weeks_total = VALUES(hits_last_two_weeks_total),
                hits_last_one_month_total = VALUES(hits_last_one_month_total),
                hits_last_three_months_total = VALUES(hits_last_three_months_total),
                hits_last_twelve_months_total = VALUES(hits_last_twelve_months_total),
                last_updated = VALUES(last_updated)
            """, (
                workbook['id'], 
                workbook['name'], 
                workbook['description'], 
                workbook['content_url'], 
                workbook['webpage_url'],
                workbook['show_tabs'], 
                workbook['size'], 
                workbook['created_at'], 
                workbook['updated_at'], 
                workbook['encrypt_extracts'],
                workbook['default_view_id'], 
                workbook['project_id'], 
                workbook['project_name'], 
                workbook['owner_id'], 
                workbook['owner_name'],
                workbook['usage_stats']['hits_total'], 
                workbook['usage_stats']['hits_last_two_weeks_total'],
                workbook['usage_stats']['hits_last_one_month_total'], 
                workbook['usage_stats']['hits_last_three_months_total'],
                workbook['usage_stats']['hits_last_twelve_months_total'], 
                workbook['date_extracted'], 
                workbook['last_updated']
            ))
    sqlconnection.commit()


### Inserting into the datasource_list table

In [14]:
# Method to insert data into the datasource_list table
def insert_datasources(sqlconnection, datasources):
    with sqlconnection.cursor() as cursor:
        for datasource in datasources:
            cursor.execute("""
                INSERT INTO datasource_list (
                    id, 
                    name, 
                    description, 
                    content_url, 
                    webpage_url, 
                    size, 
                    created_at, 
                    updated_at, 
                    encrypt_extracts,
                    project_id, 
                    project_name, 
                    owner_id, 
                    owner_name, 
                    has_extracts, 
                    is_certified, 
                    use_remote_query_agent,
                    hits_total, 
                    hits_last_two_weeks_total, 
                    hits_last_one_month_total, 
                    hits_last_three_months_total,
                    hits_last_twelve_months_total, 
                    connected_workbooks, 
                    date_extracted, 
                    last_updated
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                    name=VALUES(name),
                    description=VALUES(description),
                    content_url=VALUES(content_url),
                    webpage_url=VALUES(webpage_url), 
                    size=VALUES(size), 
                    created_at=VALUES(created_at), 
                    updated_at=VALUES(updated_at), 
                    encrypt_extracts=VALUES(encrypt_extracts), 
                    project_id=VALUES(project_id), 
                    project_name=VALUES(project_name), 
                    owner_id=VALUES(owner_id), 
                    owner_name=VALUES(owner_name), 
                    has_extracts=VALUES(has_extracts),
                    is_certified=VALUES(is_certified), 
                    use_remote_query_agent=VALUES(use_remote_query_agent), 
                    hits_total=VALUES(hits_total), 
                    hits_last_two_weeks_total=VALUES(hits_last_two_weeks_total), 
                    hits_last_one_month_total=VALUES(hits_last_one_month_total), 
                    hits_last_three_months_total=VALUES(hits_last_three_months_total), 
                    hits_last_twelve_months_total=VALUES(hits_last_twelve_months_total), 
                    connected_workbooks=VALUES(connected_workbooks), 
                    last_updated=VALUES(last_updated)
            """, (
                datasource['id'],
                datasource['name'], 
                datasource['description'], 
                datasource['content_url'], 
                datasource['webpage_url'],
                datasource['size'], 
                datasource['created_at'], 
                datasource['updated_at'], 
                datasource['encrypt_extracts'],
                datasource['project_id'], 
                datasource['project_name'], 
                datasource['owner_id'], 
                datasource['owner_name'],
                datasource['has_extracts'], 
                datasource['is_certified'], 
                datasource['use_remote_query_agent'],
                datasource['usage_stats']['hits_total'], 
                datasource['usage_stats']['hits_last_two_weeks_total'],
                datasource['usage_stats']['hits_last_one_month_total'],
                datasource['usage_stats']['hits_last_three_months_total'], 
                datasource['usage_stats']['hits_last_twelve_months_total'], 
                datasource['connected_workbooks'],
                datasource['date_extracted'], 
                datasource['last_updated']
            ))
        sqlconnection.commit()

### Inserting into the connections_list table

In [15]:
# Method to insert data into the connections_list table
def insert_connections(sqlconnection, connections):
    with sqlconnection.cursor() as cursor:
        for connection_data in connections:
            cursor.execute("""
                INSERT INTO connections_list (
                    connection_id, 
                    is_certified, 
                    certifier_id, 
                    certifier_name, 
                    certified_display_name, 
                    database_id, 
                    database_name, 
                    extract_last_refresh_time, 
                    datasource_id, 
                    datasource_name, 
                    workbook_id, 
                    workbook_name, 
                    date_extracted, 
                    last_updated
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                    is_certified=VALUES(is_certified),
                    certifier_id=VALUES(certifier_id), 
                    certifier_name=VALUES(certifier_name), 
                    certified_display_name=VALUES(certified_display_name), 
                    database_id=VALUES(database_id), 
                    database_name=VALUES(database_name), 
                    extract_last_refresh_time=VALUES(extract_last_refresh_time), 
                    datasource_id=VALUES(datasource_id), 
                    datasource_name=VALUES(datasource_name), 
                    workbook_id=VALUES(workbook_id), 
                    workbook_name=VALUES(workbook_name), 
                    last_updated=VALUES(last_updated)
            """, (
                connection_data['connection_id'], 
                connection_data['is_certified'], 
                connection_data['certifier_id'], 
                connection_data['certifier_name'], 
                connection_data['certified_display_name'], 
                connection_data['database_id'], 
                connection_data['database_name'], 
                connection_data['extract_last_refresh_time'], 
                connection_data['datasource_id'], 
                connection_data['datasource_name'], 
                connection_data['workbook_id'], 
                connection_data['workbook_name'], 
                connection_data['date_extracted'], 
                connection_data['last_updated']
            ))
        sqlconnection.commit()


## Closing the database connection

In [16]:
# Insert data into the workbook_list table
insert_workbooks(sqlconnection, workbooks_with_usage)

# Insert data into the datasource_list table
insert_datasources(sqlconnection, datasources_with_usage)

# Insert data into the connections_list table
insert_connections(sqlconnection, connects) # type: ignore

# Close the database connection
sqlconnection.close()

print("Data insertion completed successfully.")
end_time = time.time()
seconds = end_time - start_time
print(f"It took a total of {seconds} seconds to complete the program")

Data insertion completed successfully.
It took a total of 199.90578866004944 seconds to complete the program
